## AlekseevDP(DSU-4, DLL-15)_HW#6_Рекуррентные_НС(#2)_RNN#2

Задание 1. Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) и задавались следующим образом:
- x - последовательность цифр
- y1 = x1, y(i) = x(i) + x(1).
Если y(i) >= 10, то y(i) = y(i) - 10 

[Примечание: из вопросов по ДЗ на сайте:
- "Вопрос студента: "В данном случае под x1 и y1 понимается x[0] и y[0]?"
- "Ответ преподавателя: "Да"]

Задача:
- научить модель предсказывать y(i) по x(i)
- попробовать RNN, LSTM, GRU


In [1]:
import torch
import time

import random
from random import randint, seed, shuffle

In [2]:
## выведем активное устройство для обучения модели
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (DEVICE)

cpu


In [ ]:
"""Задание 1. Пример:
i    = 0   1   2   3   4   5   6   7   8   9

X[i] = 5   7   2   8   3   9   5   0   4   6

Y[i] = 5   2   7   3   8   4   0   5   9   1"""

In [3]:
# зададим количество сэмплов, а также инициализируем X и Y
samples = 100
X = torch.zeros(samples, dtype=int)
Y = torch.zeros(samples, dtype=int)

In [4]:
# сгенерируем X, предварительно зафиксировав генератор случайных чисел (для воспроизводимости результатов)
random.seed(12)
for i in range(samples):
  X[i] = randint(0, 9)

In [5]:
X[:30]

tensor([7, 4, 8, 5, 2, 6, 0, 5, 7, 4, 7, 9, 3, 8, 0, 9, 2, 7, 5, 2, 5, 3, 0, 9,
        3, 1, 8, 5, 6, 1])

In [6]:
# теперь на основании X сгенерируем Y
Y[0] = X[0]
for i in range(1, samples):
  Y[i] = X[i] + X[0]
  if Y[i] >= 10:
     Y[i] -= 10

In [7]:
Y[:30]

tensor([7, 1, 5, 2, 9, 3, 7, 2, 4, 1, 4, 6, 0, 5, 7, 6, 9, 4, 2, 9, 2, 0, 7, 6,
        0, 8, 5, 2, 3, 8])

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [10]:
X_train.shape, y_train.shape

(torch.Size([70]), torch.Size([70]))

In [11]:
X_test.shape, y_test.shape

(torch.Size([30]), torch.Size([30]))

In [12]:
BATCH_SIZE=20

In [13]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
data_train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
data_test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [14]:
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding = torch.nn.Embedding(10, 20)
        self.rnn = torch.nn.RNN(20, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)
        
    def forward(self, seq, state=None):
        embed = self.embedding(seq)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [15]:
model = RNN()

In [16]:
num_epochs = 30
LEARNING_RATE = 0.001

In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [18]:
# зададим функцию для обучения модели
def train_model():
  for ep in range(num_epochs):
    train_iters, train_passed  = 0, 0
    train_loss, train_acc = 0., 0.
    start=time.time()
    
    model.train()
    for x, y in data_train:
        optimizer.zero_grad()
        answers = model(x)
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
                
        train_acc += (answers.argmax(dim=1) == y).sum().item()
        train_iters += 1
        train_passed += len(x)
    
    test_iters, test_passed  = 0, 0
    test_loss, test_acc = 0., 0.
    model.eval()
    for x, y in data_test:
        answers = model(x)
        loss = criterion(answers, y)
        test_loss += loss.item()
        
        test_acc += (answers.argmax(dim=1) == y).sum().item()
        test_iters += 1
        test_passed += len(x)
        
    print("ep: {}, taked: {:.3f}, train_loss: {:.3f}, train_acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}".format(
        ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
        test_loss / test_iters, test_acc / test_passed)
    )

In [19]:
train_model()

ep: 0, taked: 0.020, train_loss: 2.259, train_acc: 0.214, test_loss: 2.248, test_acc: 0.200
ep: 1, taked: 0.011, train_loss: 2.255, train_acc: 0.229, test_loss: 2.244, test_acc: 0.200
ep: 2, taked: 0.011, train_loss: 2.251, train_acc: 0.229, test_loss: 2.241, test_acc: 0.233
ep: 3, taked: 0.014, train_loss: 2.246, train_acc: 0.229, test_loss: 2.237, test_acc: 0.233
ep: 4, taked: 0.012, train_loss: 2.242, train_acc: 0.243, test_loss: 2.233, test_acc: 0.233
ep: 5, taked: 0.012, train_loss: 2.237, train_acc: 0.257, test_loss: 2.229, test_acc: 0.233
ep: 6, taked: 0.013, train_loss: 2.233, train_acc: 0.257, test_loss: 2.225, test_acc: 0.233
ep: 7, taked: 0.012, train_loss: 2.229, train_acc: 0.271, test_loss: 2.221, test_acc: 0.233
ep: 8, taked: 0.011, train_loss: 2.224, train_acc: 0.271, test_loss: 2.218, test_acc: 0.233
ep: 9, taked: 0.011, train_loss: 2.220, train_acc: 0.271, test_loss: 2.214, test_acc: 0.233
ep: 10, taked: 0.011, train_loss: 2.215, train_acc: 0.286, test_loss: 2.210, tes

In [20]:
for i in range(10):
    print(f'x_test: {X_test[i]}\ny_test: {y_test[i]} \ny_pred: {model(X_test[i].unsqueeze(-1)).argmax(dim=1)}\n------------')

x_test: 2
y_test: 9 
y_pred: tensor([9])
------------
x_test: 6
y_test: 3 
y_pred: tensor([3])
------------
x_test: 9
y_test: 6 
y_pred: tensor([6])
------------
x_test: 1
y_test: 8 
y_pred: tensor([6])
------------
x_test: 6
y_test: 3 
y_pred: tensor([3])
------------
x_test: 4
y_test: 1 
y_pred: tensor([2])
------------
x_test: 6
y_test: 3 
y_pred: tensor([3])
------------
x_test: 7
y_test: 4 
y_pred: tensor([3])
------------
x_test: 3
y_test: 0 
y_pred: tensor([0])
------------
x_test: 4
y_test: 1 
y_pred: tensor([2])
------------


## Мы видим, что на небольшом исходном датасете (100 сэмплов), размере батча 20 сэмплов, небольшом количестве эпох (30) и оптимизаторе SGD точность предсказаний ячейкой RNN получается невысокой (53%). Попробуем увеличить количество эпох до 100, остальные параметры без изменений.

In [21]:
num_epochs = 100

In [22]:
model = RNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [23]:
train_model()

ep: 0, taked: 0.016, train_loss: 2.339, train_acc: 0.071, test_loss: 2.358, test_acc: 0.000
ep: 1, taked: 0.011, train_loss: 2.334, train_acc: 0.071, test_loss: 2.353, test_acc: 0.033
ep: 2, taked: 0.011, train_loss: 2.329, train_acc: 0.086, test_loss: 2.349, test_acc: 0.033
ep: 3, taked: 0.012, train_loss: 2.324, train_acc: 0.086, test_loss: 2.345, test_acc: 0.033
ep: 4, taked: 0.011, train_loss: 2.319, train_acc: 0.086, test_loss: 2.340, test_acc: 0.033
ep: 5, taked: 0.011, train_loss: 2.314, train_acc: 0.100, test_loss: 2.336, test_acc: 0.033
ep: 6, taked: 0.011, train_loss: 2.309, train_acc: 0.100, test_loss: 2.331, test_acc: 0.033
ep: 7, taked: 0.011, train_loss: 2.304, train_acc: 0.114, test_loss: 2.327, test_acc: 0.033
ep: 8, taked: 0.011, train_loss: 2.299, train_acc: 0.114, test_loss: 2.323, test_acc: 0.033
ep: 9, taked: 0.011, train_loss: 2.294, train_acc: 0.114, test_loss: 2.318, test_acc: 0.033
ep: 10, taked: 0.011, train_loss: 2.289, train_acc: 0.129, test_loss: 2.314, tes

## При увеличении кол-ва эпох до 100 точность предсказаний RNN увеличилась до 70%, что всё равно является низкой величиной. Попробуем изменить метод оптимизации с SGD на Adam. Остальные параметры без изменений.

In [24]:
num_epochs = 100

In [25]:
model = RNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [26]:
train_model()

ep: 0, taked: 0.024, train_loss: 2.268, train_acc: 0.143, test_loss: 2.187, test_acc: 0.333
ep: 1, taked: 0.012, train_loss: 2.048, train_acc: 0.571, test_loss: 2.049, test_acc: 0.600
ep: 2, taked: 0.012, train_loss: 1.853, train_acc: 0.886, test_loss: 1.918, test_acc: 0.900
ep: 3, taked: 0.013, train_loss: 1.667, train_acc: 0.957, test_loss: 1.792, test_acc: 0.900
ep: 4, taked: 0.013, train_loss: 1.487, train_acc: 0.986, test_loss: 1.672, test_acc: 0.933
ep: 5, taked: 0.013, train_loss: 1.314, train_acc: 0.971, test_loss: 1.558, test_acc: 0.933
ep: 6, taked: 0.014, train_loss: 1.150, train_acc: 0.971, test_loss: 1.453, test_acc: 0.867
ep: 7, taked: 0.013, train_loss: 0.999, train_acc: 0.971, test_loss: 1.357, test_acc: 0.867
ep: 8, taked: 0.013, train_loss: 0.864, train_acc: 0.971, test_loss: 1.270, test_acc: 0.767
ep: 9, taked: 0.013, train_loss: 0.746, train_acc: 0.971, test_loss: 1.190, test_acc: 0.733
ep: 10, taked: 0.014, train_loss: 0.643, train_acc: 0.986, test_loss: 1.118, tes

## Мы видим, что точность предсказаний 93% была достигнута ячейкой RNN уже на 5-ой эпохе, затем качество упало до 83%??? Но в целом, оптимизатор Adam справился с задачей гораздо лучше, чем SGD.

## Попробуем теперь при этих же исходных параметрах сделать предсказание с помощью ячейки LSTM.

In [27]:
from torch import nn

In [28]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        s, state = self.hidden(out)
        # predictions = self.output(state[0].squeeze())
        predictions = self.output(s)
        return predictions

In [29]:
# для сравнения возьмем те же параметры ячейки LSTM, что и в RNN
model = NeuralNetwork(nn.LSTM, 10, 20, 128, 10) 

In [30]:
num_epochs = 100
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [31]:
train_model()

ep: 0, taked: 0.037, train_loss: 2.290, train_acc: 0.171, test_loss: 2.249, test_acc: 0.367
ep: 1, taked: 0.027, train_loss: 2.227, train_acc: 0.314, test_loss: 2.199, test_acc: 0.400
ep: 2, taked: 0.026, train_loss: 2.166, train_acc: 0.443, test_loss: 2.148, test_acc: 0.433
ep: 3, taked: 0.033, train_loss: 2.100, train_acc: 0.629, test_loss: 2.093, test_acc: 0.433
ep: 4, taked: 0.025, train_loss: 2.025, train_acc: 0.643, test_loss: 2.033, test_acc: 0.433
ep: 5, taked: 0.025, train_loss: 1.938, train_acc: 0.643, test_loss: 1.970, test_acc: 0.467
ep: 6, taked: 0.025, train_loss: 1.837, train_acc: 0.629, test_loss: 1.918, test_acc: 0.367
ep: 7, taked: 0.025, train_loss: 1.727, train_acc: 0.600, test_loss: 1.873, test_acc: 0.367
ep: 8, taked: 0.031, train_loss: 1.604, train_acc: 0.686, test_loss: 1.788, test_acc: 0.467
ep: 9, taked: 0.025, train_loss: 1.464, train_acc: 0.729, test_loss: 1.683, test_acc: 0.567
ep: 10, taked: 0.024, train_loss: 1.317, train_acc: 0.786, test_loss: 1.603, tes

## Мы видим, что LSTM достигла качества предсказаний на тестовой выборке в 100%. Качество выше 90% достигнуто на 33-ей эпохе (у RNN 93% на 5-ой эпохе).

## Попробуем теперь GRU.

In [32]:
# для сравнения возьмем те же параметры ячейки GRU, что и в RNN & LSTM
model = NeuralNetwork(nn.GRU, 10, 20, 128, 10) 

In [33]:
num_epochs = 100
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [34]:
train_model()

ep: 0, taked: 0.036, train_loss: 2.276, train_acc: 0.129, test_loss: 2.214, test_acc: 0.300
ep: 1, taked: 0.027, train_loss: 2.170, train_acc: 0.414, test_loss: 2.124, test_acc: 0.567
ep: 2, taked: 0.027, train_loss: 2.070, train_acc: 0.643, test_loss: 2.034, test_acc: 0.633
ep: 3, taked: 0.030, train_loss: 1.968, train_acc: 0.700, test_loss: 1.943, test_acc: 0.600
ep: 4, taked: 0.024, train_loss: 1.862, train_acc: 0.686, test_loss: 1.849, test_acc: 0.600
ep: 5, taked: 0.025, train_loss: 1.749, train_acc: 0.686, test_loss: 1.754, test_acc: 0.600
ep: 6, taked: 0.025, train_loss: 1.627, train_acc: 0.686, test_loss: 1.659, test_acc: 0.633
ep: 7, taked: 0.026, train_loss: 1.497, train_acc: 0.700, test_loss: 1.565, test_acc: 0.633
ep: 8, taked: 0.032, train_loss: 1.361, train_acc: 0.729, test_loss: 1.467, test_acc: 0.633
ep: 9, taked: 0.028, train_loss: 1.219, train_acc: 0.814, test_loss: 1.359, test_acc: 0.667
ep: 10, taked: 0.027, train_loss: 1.077, train_acc: 0.829, test_loss: 1.246, tes

## Мы видим, что точность на тестовой выборке выше 90% достигнута ячейкой GRU на 15-ой эпохе - чуть медленнее, чем RNN (5 эпох), но значительно быстрее, чем LSTM (33 эпохи). 

## Т.е. по точности предсказаний наилучшим образом отработала LSTM (но медленнее всех). По скорости предсказаний с приемлемой точностью - RNN. При этом GRU расположилась "посередине".

## В заключение попробуем существенно увеличить количество сэмплов (со 100 до 1000 шт.) - для проверки гипотезы, что на большом датасете RNN сможет выучить закономерности быстрее, чем на малом.

In [35]:
# зададим количество сэмплов, а также инициализируем X и Y
samples = 1000
X = torch.zeros(samples, dtype=int)
Y = torch.zeros(samples, dtype=int)

In [36]:
# сгенерируем X, предварительно зафиксировав генератор случайных чисел (для воспроизводимости результатов)
random.seed(12)
for i in range(samples):
  X[i] = randint(0, 9)

In [37]:
# теперь на основании X сгенерируем Y
Y[0] = X[0]
for i in range(1, samples):
  Y[i] = X[i] + X[0]
  if Y[i] >= 10:
     Y[i] -= 10

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [39]:
# размер батча оставим прежним
BATCH_SIZE=20

In [40]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
data_train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
data_test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [41]:
model = RNN()

num_epochs = 30
LEARNING_RATE = 0.001

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [42]:
train_model()

ep: 0, taked: 0.121, train_loss: 1.783, train_acc: 0.660, test_loss: 1.224, test_acc: 0.947
ep: 1, taked: 0.114, train_loss: 0.834, train_acc: 0.993, test_loss: 0.489, test_acc: 1.000
ep: 2, taked: 0.110, train_loss: 0.310, train_acc: 0.999, test_loss: 0.178, test_acc: 1.000
ep: 3, taked: 0.107, train_loss: 0.123, train_acc: 0.999, test_loss: 0.081, test_acc: 1.000
ep: 4, taked: 0.112, train_loss: 0.065, train_acc: 0.999, test_loss: 0.047, test_acc: 1.000
ep: 5, taked: 0.106, train_loss: 0.042, train_acc: 0.999, test_loss: 0.032, test_acc: 1.000
ep: 6, taked: 0.120, train_loss: 0.030, train_acc: 0.999, test_loss: 0.023, test_acc: 1.000
ep: 7, taked: 0.115, train_loss: 0.023, train_acc: 0.999, test_loss: 0.018, test_acc: 1.000
ep: 8, taked: 0.112, train_loss: 0.018, train_acc: 0.999, test_loss: 0.014, test_acc: 1.000
ep: 9, taked: 0.129, train_loss: 0.015, train_acc: 0.999, test_loss: 0.012, test_acc: 1.000
ep: 10, taked: 0.118, train_loss: 0.012, train_acc: 0.999, test_loss: 0.010, tes

## Гипотеза более быстрого обучения RNN на большой выборке подтвердилась - точность 100% на тестовой выборке достигнута уже на 2-ой эпохе, т.е. количество примеров для обучения (в случае несложной закономерности - такой, как в данной задаче) существенно улучшило качество и скорость предсказаний RNN.